In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
from aggregate_scoring import ScoringCriterion, AggregateScoringSystem, CommunityTransportationOptions, QualityEducation, DesirableUndesirableActivities, StableCommunities,HousingNeedsCharacteristics

/opt/homebrew/anaconda3/lib/python3.12/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
kwargs = {
    # --- CommunityTransportationOptions ---
    "transit_df": pd.read_csv("../../data/raw/scoring_indicators/community_trans_options_sites/georgia_transit_locations_with_hub.csv"),

    # --- DesirableUndesirableActivities ---
    "rural_gdf_unary_union": gpd.read_file("../../data/raw/shapefiles/USDA_Rural_Housing_by_Tract_7054655361891465054/USDA_Rural_Housing_by_Tract.shp").to_crs("EPSG:4326").unary_union,
    "desirable_csv": pd.read_csv("../../data/processed/scoring_indicators/desirable_undesirable_activities/desirable_activities_google_places_v3.csv"),
    "grocery_csv": pd.read_csv("../../data/processed/scoring_indicators/desirable_undesirable_activities/desirable_activities_google_places_v3.csv"),
    "usda_csv": pd.read_csv("../../data/raw/scoring_indicators/desirable_undesirable_activities/usda/food_access_research_atlas.csv", dtype={'CensusTract': str}),
    "tract_shapefile": gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp"),
    "undesirable_csv": pd.read_csv("../../data/processed/scoring_indicators/desirable_undesirable_activities/undesirable_hsi_tri_cdr_rcra_frs_google_places.csv"),

    # --- QualityEducation ---
    "school_df": pd.read_csv("../../data/processed/scoring_indicators/quality_education/Option_C_Scores_Eligibility_with_BTO.csv"),
    "school_boundary_gdfs": [
    gpd.read_file("../../data/raw/shapefiles/quality_education/Administrative.geojson").to_crs("EPSG:4326"),
    gpd.read_file("../../data/raw/shapefiles/quality_education/APSBoundaries.json").to_crs("EPSG:4326"),
    gpd.read_file("../../data/raw/shapefiles/quality_education/DKE.json").to_crs("EPSG:4326"),
    gpd.read_file("../../data/raw/shapefiles/quality_education/DKM.json").to_crs("EPSG:4326"),
    gpd.read_file("../../data/raw/shapefiles/quality_education/DKBHS.json").to_crs("EPSG:4326")
    ],       
    "state_avg_by_year": {
        "elementary": {
            2018: 77.8,
            2019: 79.9
        },
        "middle": {
            2018: 76.2,
            2019: 77
        },
        "high": {
            2018: 75.3,
            2019: 78.8
        }
    },

    # --- StableCommunities ---
    "indicators_df": pd.read_csv("../../data/processed/scoring_indicators/stable_communities/stable_communities_2024_processed_v3.csv"),
    "tracts_shp": gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp").to_crs("EPSG:4326"),
    
    # --- HousingNeedsCharacteristics ---
    "census_tract_data": pd.read_csv("../../data/processed/scoring_indicators/housing_needs/merged_housing_data.csv"),
    "tracts_gdf": gpd.read_file("../../data/raw/shapefiles/HousingNeeds/tl_2020_13_tract/tl_2020_13_tract.shp").to_crs("EPSG:4326"),
    #"revitalization_score": 4,
    "in_qct": False  # Required for housing need eligibility
} 

/var/folders/w2/jx7kgvv14cx4km6tl2rymsmw0000gn/T/ipykernel_38324/3108687093.py:6: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  "rural_gdf_unary_union": gpd.read_file("../../data/raw/shapefiles/USDA_Rural_Housing_by_Tract_7054655361891465054/USDA_Rural_Housing_by_Tract.shp").to_crs("EPSG:4326").unary_union,


# Compute Scores

In [4]:
site_latitude = 33.856192
site_longitude = -84.347348

### Community Transportation Options Score

In [5]:
community_transportation_options_score = CommunityTransportationOptions(site_latitude, site_longitude, **kwargs).calculate_score()
print(f"Community Transportation Options Score: {community_transportation_options_score}")

Community Transportation Options Score: 4.0


### Desirable/Undesirable Activities Score

In [6]:
desirable_undesirable_activities_score = DesirableUndesirableActivities(site_latitude, site_longitude, **kwargs).calculate_score()
print(f"Desirable Undesirable Activities Score: {desirable_undesirable_activities_score}")

Desirable Undesirable Activities Score: 20


### Quality Education Areas Score

In [7]:
quality_education_score = QualityEducation(site_latitude, site_longitude, **kwargs).calculate_score()
print(f"Quality Education Score: {quality_education_score}")

Quality Education Score: 3


### Stable Communities Score

In [8]:
stable_communities_score = StableCommunities(site_latitude, site_longitude, **kwargs).calculate_score()
print(f"Stable Communities Score: {stable_communities_score}")

Stable Communities Score: 9


### Total Location Score

In [9]:
total_location_score = community_transportation_options_score + desirable_undesirable_activities_score + quality_education_score + stable_communities_score 
print(f"Total Location Score: {total_location_score}")

Total Location Score: 36.0
